Application in which we read and process altimetry measurements using `diva`.

In [1]:
import os
import re
import glob
import logging
import numpy as np
from matplotlib import rcParams
from mpl_toolkits.basemap import Basemap
import pydiva2d
import netCDF4
import divaaltimetry
import matplotlib.pyplot as plt

If you get a `ModuleNotFoundError` message, ensure that the module `pydiva2d` is available in the Python path.

# Setup 

## Logging configuration

The *logging* is already configured in `pydiva2d`.<br>
Replace 'DEBUG' by 'INFO', 'WARNING' or 'ERROR'.

In [2]:
logger = logging.getLogger('divaAltimetry')
logger.setLevel(logging.DEBUG)

## Files and directories

Set the path to the Diva installation you want and the main input files.   
A different file will be used for the mesh generation.

In [3]:
divadir = "/home/ctroupin/Software/DIVA/DIVA-diva-4.7.1"
projectdir = "/home/ctroupin/Projects/Altimetry-Interpolation/"
coastfile = os.path.join(projectdir, "diva/coast.cont")
datadir = os.path.join(projectdir, "data/SEALEVEL_MED_PHY_L3_REP_OBSERVATIONS_008_049/diva/")
paramfile = os.path.join(projectdir, "diva/param.par")
paramfilemesh = os.path.join(projectdir, "diva/param.par.mesh")
outputdir = os.path.join(projectdir, "results/SEALEVEL_MED_PHY_L3_REP_OBSERVATIONS_008_049/diva2D/")
figdir = "../figures/diva2d/"
if not os.path.exists(divadir):
    logger.error("Diva directory doesn't exist")
if not os.path.exists(outputdir):
    os.makedirs(outputdir)
    logger.debug("Create output directory")
if not os.path.exists(figdir):
    os.makedirs(figdir)
    logger.debug("Create figure directory")

## Matplotlib options

In [4]:
rcParams.update({'font.size': 14, 'figure.dpi': 300, 'savefig.bbox': 'tight'})
coordinates = (-6.75, 36.001, 30, 48.)
meridians = np.arange(-8., 40., 8.)
parallels = np.arange(30., 50., 4.5)

### Projection
Will be used for the plots.

In [5]:
m = Basemap(projection='merc',
            llcrnrlon=coordinates[0], llcrnrlat=coordinates[2],
            urcrnrlon=coordinates[1], urcrnrlat=coordinates[3],
            lat_ts=0.5 * (coordinates[2] + coordinates[3]), resolution='i')

## Directories
Create variables storing the Diva directories and files.

In [6]:
DivaDirs = pydiva2d.DivaDirectories(divadir)
DivaFiles = pydiva2d.Diva2Dfiles(DivaDirs.diva2d)

# Analysis
## Create mesh
As the domain is always the same (surface level), we only create the mesh once at the beginning.

In [7]:
mesh2d = pydiva2d.Diva2DMesh().make(divadir, contourfile=coastfile, paramfile=paramfilemesh)
mesh2d.describe()

Number of nodes: 6704
Number of interfaces: 16293
Number of elements: 9496


## Diva calculation
We use another parameter file, loop over the data files and copy the outputs (netCDF) in the specified directory.

In [9]:
datafilelist = sorted(glob.glob(os.path.join(datadir, 'data_201402*.dat')))
logger.info("Working on {} files".format(len(datafilelist)))

datepattern = re.compile(r'data_(\d{8})_(\d{8})')

for datafile in datafilelist:
    
    print(datafile)
    outputfile = "".join((os.path.basename(datafile).split('.')[0], '.nc'))
    figname = os.path.basename(datafile).split('.')[0]
    
    match = datepattern.search(os.path.basename(datafile))
    if match:
        d1 = "-".join((match.group(1)[:4], match.group(1)[4:6], match.group(1)[6:8]))
        d2 = "-".join((match.group(2)[:4], match.group(2)[4:6], match.group(2)[6:8]))
        figtitle = "$-$".join((d1, d2)) 
    logger.info("Output file: {0}".format(outputfile))
    
    results2d = pydiva2d.Diva2DResults().make(divadir, datafile=datafile,
                                              paramfile=paramfile, 
                                              contourfile=coastfile,
                                              outputfile=os.path.join(outputdir, outputfile))
    
    
    # Make plot
    SLA = divaaltimetry.AltimetryField().from_diva2d_file(os.path.join(outputdir, outputfile))
    SLA.add_to_plot(figname=os.path.join(figdir, figname), figtitle=figtitle, m=m,
                    meridians=meridians, parallels=parallels,
                    vmin=-0.2, vmax=0.2,
                    cmap=plt.cm.RdYlBu_r)    
    
logger.info("Summary:")
logger.info("Figures available in {}".format(os.path.abspath(figdir)))
logger.info("Results available in {}".format(os.path.abspath(outputdir)))

/home/ctroupin/Projects/Altimetry-Interpolation/data/SEALEVEL_MED_PHY_L3_REP_OBSERVATIONS_008_049/diva/data_20140201_20140221.dat
/home/ctroupin/Projects/Altimetry-Interpolation/data/SEALEVEL_MED_PHY_L3_REP_OBSERVATIONS_008_049/diva/data_20140202_20140222.dat
/home/ctroupin/Projects/Altimetry-Interpolation/data/SEALEVEL_MED_PHY_L3_REP_OBSERVATIONS_008_049/diva/data_20140203_20140223.dat
/home/ctroupin/Projects/Altimetry-Interpolation/data/SEALEVEL_MED_PHY_L3_REP_OBSERVATIONS_008_049/diva/data_20140204_20140224.dat
/home/ctroupin/Projects/Altimetry-Interpolation/data/SEALEVEL_MED_PHY_L3_REP_OBSERVATIONS_008_049/diva/data_20140205_20140225.dat
/home/ctroupin/Projects/Altimetry-Interpolation/data/SEALEVEL_MED_PHY_L3_REP_OBSERVATIONS_008_049/diva/data_20140206_20140226.dat
/home/ctroupin/Projects/Altimetry-Interpolation/data/SEALEVEL_MED_PHY_L3_REP_OBSERVATIONS_008_049/diva/data_20140207_20140227.dat
/home/ctroupin/Projects/Altimetry-Interpolation/data/SEALEVEL_MED_PHY_L3_REP_OBSERVATIONS_